In [7]:
%load_ext autoreload
%autoreload 2

import os 
import shutil
import numpy as n
from datetime import date
from matplotlib import pyplot as plt
import time
import napari
from datetime import datetime

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
# path to wherever the s2p-lbm repository is cloned on your computer
# CHANGE THIS!!!!
repo_path = '/home/ali/packages/s2p-lbm/'#os.path.dirname(os.path.abspath(""))
os.chdir(repo_path)

from suite3d.job import Job
from suite3d import lbmio, utils, ui
from suite3d import tiff_utils as tfu

In [9]:
results_dir = '/mnt/md0/s3d-benchmarking/results/corr_map'

In [10]:
# This is the original job that I used for benchmarking
original_job = Job('/mnt/md0/s3d-benchmarking','benchmark-corr_map',create=False, verbosity = 3)

   Found and loaded params from /mnt/md0/s3d-benchmarking/s3d-benchmark-corr_map/params.npy


In [34]:
# here you can create your own job. Instead of re-running all the previous processing steps
# up to the correlation map, I've added a function to just copy the relevant files from a 
# parent job. This way you can do whatever you want on your own job without overwriting the 
# benchmarking data in the original job

In [33]:
job = Job('/mnt/md0/sam', 'corr_map_parallelized', create=True, overwrite=True, 
          parent_job=original_job, copy_parent_dirs=('registered_fused_data',))

Job directory /mnt/md0/sam/s3d-corr_map_parallelized already exists
Loading job directory for corr_map_parallelized in /mnt/md0/sam
   Loading dirs 
      Found dir registered_data
      Found dir summary
      Found dir iters
   Found and loaded params from /mnt/md0/s3d-benchmarking/s3d-benchmark-corr_map/params.npy
   Copied init pass and parameters from parent job
   Copying dir registered_fused_data from parent job
      Found dir /mnt/md0/sam/s3d-corr_map_parallelized/registered_fused_data with tag registered_fused_data
   Updated main params file


In [35]:
mov = job.get_registered_movie('registered_fused_data', 'fused')

In [36]:
job.params['conv_filt_type'] = 'gaussian'
job.params['conv_filt_xy'] = 1.0
job.params['conv_filt_z'] = 1.0
job.params['npil_filt_type'] = 'unif'
job.params['npil_filt_xy'] = 5.0
job.params['npil_filt_z']=  1.5

job.params['sdnorm_exp']= 0.8
job.params['intensity_thresh'] : 0.2

job.params['t_batch_size'] = 300
job.params['n_proc_corr'] = 70
job.params['mproc_batchsize'] = 5


crop = ((0,18), (100,1100), (100, 950))
mov_crop = mov[crop[0][0]:crop[0][1], :,crop[1][0]:crop[1][1], 
               crop[2][0]:crop[2][1]]

In [37]:
%%time
tic = time.time()
(vmap, mean_img, max_img), __, __ = job.calculate_corr_map(mov = mov)
toc = time.time()

   Updated main params file
   Created dir /mnt/md0/sam/s3d-corr_map_parallelized/mov_sub with tag mov_sub
   Saving mov_sub to /mnt/md0/sam/s3d-corr_map_parallelized/mov_sub
   Using conv_filt: gaussian, 1.00, 1.00
   Using np_filt: unif, 1.50, 5.00
   Created files and dirs for 1 batches
      Running batch 1 of 1
         Will process 300 frames (0-300, t_batch_size: 300)
      Loaded and swapped, idx 0 to 300
      Calculating corr map
         Rolling mean filter
         Stdev over time
         Sharr creation
         Sub and conv
         Vmap
   Calculated corr map in 446.67 seconds
      Saving to /mnt/md0/sam/s3d-corr_map_parallelized/iters/batch0000
CPU times: user 1min 45s, sys: 5min 27s, total: 7min 12s
Wall time: 7min 50s


In [45]:
timings = {'corr_map'    : toc - tic}
outputs = {'vmap'        : vmap,
           'mean_img'    : mean_img}
repo_status = utils.get_repo_status(repo_path)

In [46]:
utils.benchmark(results_dir, outputs, timings, repo_status)

                                         Baseline        |      Current         | 
Branch:                             dev                  | dev                  | 
Last commit hash:                   4f756530fe7b1a8b8a8b | 3b23e046a2b2acc29198 | 
Last commit summ:                   benchmarks for init, | benchmarking tools e | 
Dirty :                             True                 | True                 | 
    
Timings (s) 
corr_map                                         395.132 |              470.270 | 

Outputs: 
vmap                                                     |                      |  mismatch: 0 / 36835470 (100.00000 % match) 
           shape:                   (30, 1239, 991)      | (30, 1239, 991)      | 
           mean:                                   2.053 |                2.053 |
           std:                                    0.842 |                0.842 | 
mean_img                                                 |                      |  mismatch: 0 / 3